<a href="https://colab.research.google.com/github/Dohun-Im/Biz-Lab/blob/main/%EC%9E%90%EB%B3%B8%EB%B3%80%EB%8F%99%EC%82%AC%ED%95%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opendartreader

In [ ]:
pip install --upgrade opendartreader

In [ ]:
import pandas as pd
import re
import glob
import requests
import xml.etree.ElementTree as ET
import zipfile
import io
import csv
import os, re, codecs
import json
import numpy as np
import datetime
try:
    from io import BytesIO
    from pandas import json_normalize
    from pandas import DataFrame as df
except ImportError:
    from pandas.io.json import json_normalize
import OpenDartReader
from bs4 import BeautifulSoup
%matplotlib inline
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
api_key = '****'
dart = OpenDartReader(api_key)

In [ ]:
os.chdir(r'/content/drive/MyDrive/주석txt/')
st = pd.read_excel(r'/content/drive/MyDrive/주석txt/회사정보 list.xlsx')
st = st['symbol'].str.replace(r'[^\d]','')
st
# st1 = st[0:600]
# st2 = st[600:1200]
# st3 = st[1200:]


In [ ]:
#연결재무제표 주석 크롤링
st2 = ['035900','085310']
for m in st2:
  try:
    report = dart.list(m, start = '2011',end= '2021.4.29', kind='A',final=True) 
    report_사업보고서 = report[report['report_nm'].str.contains('사업')]
    report2 = report_사업보고서[['corp_name','stock_code','rcept_no','rcept_dt','report_nm']]
    n = report2['rcept_no']

    
    dt = report2['report_nm']
    dt = ''.join(dt) 
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')
    dt = re.sub(korean,'',dt)
    dt = re.sub(r'[^a-zA-Z0-9]', '', dt)
    dt = re.sub('12','',dt)
    def list_chunk(lst, n):
            return [lst[i:i+n] for i in range(0, len(lst), n)]
    dt = list_chunk(dt, 4)
    for j,d in zip(n,dt):
      sub_report = dart.sub_docs(j)
      try:
        url = sub_report.loc[sub_report['title'].str.contains('자본금'), 'url'].values[0]
        soup = BeautifulSoup(requests.get(url).text)
        soup = soup.text  
        with open('/content/drive/MyDrive/자본금txt/txt/'+m+d+j+'.txt', 'w', encoding= 'utf-8') as file:
          file.write(soup)
        # print(url)
      except IndexError:
        continue
     
  #   continue
  except IndexError:
    continue
  except: {"status":"013","message":"조회된 데이타가 없습니다."}
  continue

In [ ]:
os.chdir(r'/content/drive/MyDrive/자본금txt/txt')
name = os.listdir(r'/content/drive/MyDrive/자본금txt/txt')
# del name[0]
# name_1 = name[0:4000]

# name_2 = name[4000:8000]
# name_3 = name[8000:]
# name_1

In [ ]:
#전환사채 table
os.chdir(r'/content/drive/MyDrive/자본금txt/txt')
for o in name:
    rcept_no = o[10:24]
    fyr = o[6:10]
    symbol = 'A'+ o[0:6]
    file_name = o[0:24]

    f = open(r'/content/drive/MyDrive/자본금txt/txt/'+ o, 'r',  encoding = 'utf-8') 
    web = f.read()
    web = re.sub(r'\xa0', '',web)
    web = re.sub(' ', '', web)
    x = '미상환전환사채발행현황'
    if web.find(x) != -1:
      n = web.find(x)
      y = '미상환신주인수권부사채등발행현황'
      z = '미상환신주인수권부사채발행현황'
      if web.find(y) != -1:
        nn = web.find(y)-1
      elif web.find(z) != -1:
        nn = web.find(z)-1
      elif web.find(y) == -1:  
        nn = len(web)  
      elif web.find(z) == -1:
        nn = len(web)
        
      web = web[n:nn]
      
      web = re.sub('전환조건','',web)
      web = re.sub('미상환사채','',web) 
      web = re.sub('행사조건','',web)
      web = re.sub('미행사신주인수권부사채','',web)
      web = re.sub('전자등록','',web)
      web = re.sub('기준일:','',web)
      web = re.sub('비고','',web)
        # web = re.sub('-','',web)
              
      characters = ')(%~'
      for t in range(len(characters)):
        web = web.replace(characters[t],'')
          
      web_temp = web

      regex2 = re.compile('(.*){}'.format(re.escape('\n'))) 
      web_temp2 = regex2.findall(web_temp)
      web_temp2 = list(filter(None, web_temp2))
      def list_chunk(lst, n):
        return [lst[i:i+n] for i in range(0, len(lst), n)]  
        
      if '회차' in web_temp2:
          # web_temp2.insert(14,'비고')
        time = web_temp2[1]
        unit = web_temp2[2]
        del web_temp2[0:14]
          
        list_chunked = list_chunk(web_temp2, 12)
        df = pd.DataFrame(list_chunked)
        df.columns = ['종류\구분','회차','발행일','만기일','권면총액','전환대상주식의종류','전환청구가능기간','전환비율','전환가액','미상환권면총액','전환주식가능수','비고']
        df['단위'] = pd.Series([unit])
        df['기준일'] = pd.Series([time])
        df['symbol'] = pd.Series([symbol])
        df = df[['symbol','기준일','단위','종류\구분','발행일','만기일','권면총액','전환대상주식의종류','전환청구가능기간','전환비율','전환가액','미상환권면총액','전환주식가능수','비고']]
        df = df.replace({'\-':'', '\,':'','\.':''}, regex=True)
        df = df.replace({'년':'', '월':'','일':''}, regex=True)
        df['만기일'].replace('',np.nan, inplace=True) #이걸 기준으로 밑에 밑의 오류들을 지웠는데 뒤에 BW공시 있으면 작동안됨
        df.dropna(subset=['만기일'], inplace=True)
        df[['기준일','발행일','만기일']] = df[['기준일','발행일','만기일']].astype('str')
        df['기준일']= pd.to_datetime(df['기준일']).dt.date
        df['발행일']= pd.to_datetime(df['발행일']).dt.date
        df['만기일']= pd.to_datetime(df['만기일']).dt.date
          # df['전환청구가능기간']=pd.to_datetime(df['전환청구가능기간'])
          
          # df[['symbol','기준일','단위']].replace('',np.nan, inplace=True)
          # df.fillna(method='ffill')
        df.to_excel('/content/drive/My Drive//자본금txt/CBtable/'+file_name+'.xlsx')
        

      elif '회차' not in web_temp2:
        # web_temp2.insert(13,'비고')
        time = web_temp2[1]
        unit = web_temp2[2]
        del web_temp2[0:13]

       
        list_chunked = list_chunk(web_temp2, 11)
        df = pd.DataFrame(list_chunked)
        df.columns = ['종류\구분','발행일','만기일','권면총액','전환대상주식의종류','전환청구가능기간','전환비율','전환가액','미상환권면총액','전환주식가능수','비고']
        df['단위'] = pd.Series([unit])
        df['기준일'] = pd.Series([time])
        df['symbol'] = pd.Series([symbol])
        df = df[['symbol','기준일','단위','종류\구분','발행일','만기일','권면총액','전환대상주식의종류','전환청구가능기간','전환비율','전환가액','미상환권면총액','전환주식가능수','비고']]
        df = df.replace({'\-':'', '\,':'','\.':''}, regex=True)
        df = df.replace({'년':'', '월':'','일':''}, regex=True)
        df['만기일'].replace('',np.nan, inplace=True)
        df.dropna(subset=['만기일'], inplace=True)
        df[['기준일','발행일','만기일']] = df[['기준일','발행일','만기일']].astype('str')
        df['기준일']= pd.to_datetime(df['기준일']).dt.date
        df['발행일']= pd.to_datetime(df['발행일']).dt.date
        df['만기일']= pd.to_datetime(df['만기일']).dt.date
        df.to_excel('/content/drive/My Drive///자본금txt/CBtable/'+file_name+'.xlsx')
      
    else:
      pass
  
  



   
     


    

In [ ]:
print(df.info())

In [ ]:
#신주인수권부사채 table
os.chdir(r'/content/drive/MyDrive/자본금txt/txt')
for o in name:
    rcept_no = o[10:24]
    fyr = o[6:10]
    symbol = 'A'+ o[0:6]
    file_name = o[0:24]

    f = open(r'/content/drive/MyDrive/자본금txt/txt/'+ o, 'r',  encoding = 'utf-8') 
    web = f.read()
    web = re.sub(r'\xa0', '',web)
    web = re.sub(' ', '', web)
    x = '미상환신주인수권부사채등발행현황'
    if web.find(x) != -1:
      n = web.find(x)
      y = '미상환전환형조건부자본증권발행현황'
      if web.find(y) != -1:
        nn = web.find(y)-1
      elif web.find(y) == -1:
        nn = len(web)
        
      web = web[n:nn]
      
      web = re.sub('전환조건','',web)
      web = re.sub('미상환사채','',web) 
      web = re.sub('행사조건','',web)
      web = re.sub('미행사신주인수권부사채','',web)
      web = re.sub('전자등록','',web)
      web = re.sub('기준일:','',web)
      web = re.sub('비고','',web)
      # web = re.sub('-','',web)
              
      characters = ')(%~'
      for t in range(len(characters)):
        web = web.replace(characters[t],'')
          
      web_temp = web

      regex2 = re.compile('(.*){}'.format(re.escape('\n'))) 
      web_temp2 = regex2.findall(web_temp)
      web_temp2 = list(filter(None, web_temp2))
      def list_chunk(lst, n):
        return [lst[i:i+n] for i in range(0, len(lst), n)]  
        
      if '회차' in web_temp2:
          # web_temp2.insert(14,'비고')
        time = web_temp2[1]
        unit = web_temp2[2]
        del web_temp2[0:14]
          
        list_chunked = list_chunk(web_temp2, 12)
        df = pd.DataFrame(list_chunked)
        df.columns = ['종류\구분','회차','발행일','만기일','권면총액','전환대상주식의종류','전환청구가능기간','전환비율','전환가액','미상환권면총액','전환주식가능수','비고']
        df['단위'] = pd.Series([unit])
        df['기준일'] = pd.Series([time])
        df['symbol'] = pd.Series([symbol])
        df = df[['symbol','기준일','단위','종류\구분','발행일','만기일','권면총액','전환대상주식의종류','전환청구가능기간','전환비율','전환가액','미상환권면총액','전환주식가능수','비고']]
        df = df.replace({'\-':'', '\,':'','\.':''}, regex=True)
        df = df.replace({'년':'', '월':'','일':''}, regex=True)
        df['만기일'].replace('',np.nan, inplace=True) #이걸 기준으로 밑에 밑의 오류들을 지웠는데 뒤에 BW공시 있으면 작동안됨
        df.dropna(subset=['만기일'], inplace=True)
        df[['기준일','발행일','만기일']] = df[['기준일','발행일','만기일']].astype('str')
        df['기준일']= pd.to_datetime(df['기준일']).dt.date
        df['발행일']= pd.to_datetime(df['발행일']).dt.date
        df['만기일']= pd.to_datetime(df['만기일']).dt.date
          # df['전환청구가능기간']=pd.to_datetime(df['전환청구가능기간'])
          
          # df[['symbol','기준일','단위']].replace('',np.nan, inplace=True)
          # df.fillna(method='ffill')
        df.to_excel('/content/drive/My Drive//자본금txt/BWtable/'+file_name+'.xlsx')
        

      elif '회차' not in web_temp2:
        # web_temp2.insert(13,'비고')
        time = web_temp2[1]
        unit = web_temp2[2]
        del web_temp2[0:13]

       
        list_chunked = list_chunk(web_temp2, 11)
        df = pd.DataFrame(list_chunked)
        df.columns = ['종류\구분','발행일','만기일','권면총액','행사대상주식의종류','신주인수권행사가능기간','행사비율','행사가액','미행사권면총액','행사가능주식수','비고']
        df['단위'] = pd.Series([unit])
        df['기준일'] = pd.Series([time])
        df['symbol'] = pd.Series([symbol])
        df = df[['symbol','기준일','단위','종류\구분','발행일','만기일','권면총액','행사대상주식의종류','신주인수권행사가능기간','행사비율','행사가액','미행사권면총액','행사가능주식수','비고']]
        df = df.replace({'\-':'', '\,':'','\.':''}, regex=True)
        df = df.replace({'년':'', '월':'','일':''}, regex=True)
        df['만기일'].replace('',np.nan, inplace=True)
        df.dropna(subset=['만기일'], inplace=True)
        df[['기준일','발행일','만기일']] = df[['기준일','발행일','만기일']].astype('str')
        df['기준일']= pd.to_datetime(df['기준일']).dt.date
        df['발행일']= pd.to_datetime(df['발행일']).dt.date
        df['만기일']= pd.to_datetime(df['만기일']).dt.date
        df.to_excel('/content/drive/My Drive///자본금txt/BWtable/'+file_name+'.xlsx')
      
    else:
      pass
# pd.set_option('display.max_columns', 500)
# print(df)
  